WRDS Options Cleaner

**Objective:**

1. **Audit:** Peek inside massive CSV files to see what dates/tickers they contain without loading the whole file.
    
2. **Filter:** Extract only the tickers we care about (e.g., `SPX` for S&P 500) to create a manageable dataset.
    
3. **Compress:** Save the cleaned data to `.parquet` format (much faster and smaller than CSV).
    

**Target Ticker:** `SPX` (Standard S&P 500 options) - Best for Heston calibration.

## Imports and Setup

In [1]:
import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime

# --- CONFIGURATION ---
# Update this path to where your screenshot folder is
RAW_DATA_PATH = r"G:\My Drive\00)Quant_Scripts\WRDS Data\Returns\Options" 

# The Ticker you want to extract for your Math Playground
# Note: In OptionMetrics, S&P 500 is often 'SPX' or ticker_id '108105'
TARGET_TICKER = 'SPX' 

# Output filename
OUTPUT_FILE = "SPX_Options_Cleaned_2025.parquet"

In [2]:
def audit_files(directory):
    files = glob.glob(os.path.join(directory, "*")) # grab all files
    
    print(f"{'File Name':<40} | {'Size (GB)':<10} | {'Columns detected'}")
    print("-" * 100)
    
    valid_csvs = []
    
    for f in files:
        if os.path.isdir(f): continue # skip folders
        
        # Get size
        size_gb = os.path.getsize(f) / (1024**3)
        
        # Peek at content (try/except for non-csv garbage files)
        try:
            # Read only first 2 rows to check columns
            df_peek = pd.read_csv(f, nrows=2)
            cols = ", ".join(df_peek.columns[:4]) + "..." # Show first 4 cols
            print(f"{os.path.basename(f):<40} | {size_gb:<10.2f} | {cols}")
            valid_csvs.append(f)
        except Exception as e:
            print(f"{os.path.basename(f):<40} | {size_gb:<10.2f} | [Error reading file]")
            
    return valid_csvs

# Run the Audit
print("Scanning Directory...")
csv_files = audit_files(RAW_DATA_PATH)

Scanning Directory...
File Name                                | Size (GB)  | Columns detected
----------------------------------------------------------------------------------------------------
OptionVolume_1996-2025.csv               | 4.09       | secid, date, cp_flag, index_flag...
OptionsVolatilitySurface_3.csv           | 42.55      | secid, date, days, delta...
OptionsVolatilitySurface_4.csv           | 32.28      | secid, date, days, delta...
OptionsVolatilitySurface_2.csv           | 38.05      | secid, date, days, delta...
OptionsVolatilitySurface_1.csv           | 27.51      | secid, date, days, delta...
OptionsVolatilitySurface_2024-2025.csv   | 41.92      | secid, date, days, delta...
v1zilpeskdggmanu.csv                     | 43.13      | secid, date, days, delta...
nbbtnjkbfwepkxdq.csv                     | 43.96      | secid, date, days, delta...
OptionsVolatilitySurface1996-2025_Filtered.csv | 19.91      | secid, date, days, delta...


In [7]:
def process_and_filter(file_list, target_ticker):
    filtered_chunks = []
    total_rows_saved = 0
    
    # Columns we actually need (Change these based on what the Audit in Cell 3 shows!)
    # Standard OptionMetrics names:
    keep_cols = ['date', 'exdate', 'cp_flag', 'strike_price', 'best_bid', 'best_offer', 'impl_volatility', 'delta', 'gamma', 'vega', 'ticker']
    
    print(f"\nStarting Extraction for ticker: {target_ticker}")
    
    for file_path in file_list:
        print(f"Processing {os.path.basename(file_path)}...")
        
        # Create an iterator to read in chunks
        # chunksize=1_000_000 means read 1 million rows at a time
        chunk_iter = pd.read_csv(file_path, chunksize=1_000_000, low_memory=False)
        
        for i, chunk in enumerate(chunk_iter):
            # 1. Standardize Column Names (Lowercase)
            chunk.columns = [c.lower() for c in chunk.columns]
            
            # 2. Filter for Target
            # Check if 'ticker' column exists, otherwise skip or guess
            if 'ticker' in chunk.columns:
                mask = chunk['ticker'] == target_ticker
                target_data = chunk[mask].copy()
                
                if not target_data.empty:
                    # 3. Keep only useful columns (intersection of what we want and what exists)
                    available_cols = list(set(keep_cols) & set(target_data.columns))
                    filtered_chunks.append(target_data[available_cols])
                    total_rows_saved += len(target_data)
                    
            if i % 10 == 0:
                print(f"  > Scanned chunk {i}... Found {total_rows_saved} rows so far.")

    if not filtered_chunks:
        print("No data found for this ticker!")
        return None
        
    print("Concatenating all chunks...")
    full_df = pd.concat(filtered_chunks, ignore_index=True)
    
    # Final Cleanup
    # Convert dates to datetime objects
    if 'date' in full_df.columns: pd.to_datetime(full_df['date'], format='%Y%m%d', errors='ignore')
    if 'exdate' in full_df.columns: pd.to_datetime(full_df['exdate'], format='%Y%m%d', errors='ignore')
    
    # Sort
    full_df = full_df.sort_values(by=['date', 'strike_price'])
    
    return full_df



In [ ]:
if df_clean is not None:
    print(f"Saving {len(df_clean)} rows to {OUTPUT_FILE}...")
    
    # Save to Parquet (requires pyarrow or fastparquet installed)
    # pip install pyarrow
    df_clean.to_parquet(OUTPUT_FILE, index=False)
    
    print("Success! Data Forge Complete.")
    
    # Quick Preview
    print(df_clean.head())
else:
    print("Failed to generate data.")

In [3]:
import os
import pandas as pd
import csv

def get_csv_date_range(file_path, date_col_name='date'):
    """
    Efficiently reads the first and last recorded date in a large CSV
    without loading the entire file.
    """
    try:
        # 1. Get the Header and First Date (Start)
        # Read just the first 5 rows
        df_head = pd.read_csv(file_path, nrows=5)
        
        # Normalize column names to lower case to find 'date'
        df_head.columns = [c.lower() for c in df_head.columns]
        
        if date_col_name not in df_head.columns:
            return "Col Not Found", "Col Not Found"
            
        start_date = df_head[date_col_name].iloc[0]
        
        # 2. Get the Last Date (End) using file seeking
        # This jumps to the end of the file byte-by-byte to read the tail
        with open(file_path, 'rb') as f:
            f.seek(0, os.SEEK_END) # Jump to end
            file_size = f.tell()
            
            # Seek back ~4096 bytes (should be enough for last few rows)
            seek_offset = min(file_size, 4096) 
            f.seek(-seek_offset, os.SEEK_END)
            
            # Read tail and decode
            tail_lines = f.read().decode('utf-8', errors='ignore').splitlines()
            
            # The last line might be empty or incomplete, grab the last valid one
            last_line = tail_lines[-1] if tail_lines[-1] else tail_lines[-2]
            
            # Parse the CSV string manually
            reader = csv.reader([last_line])
            last_row = list(reader)[0]
            
            # We need to map the 'date' column index from the header
            col_idx = df_head.columns.get_loc(date_col_name)
            end_date = last_row[col_idx]
            
        return start_date, end_date

    except Exception as e:
        return "Error", str(e)

# --- RUN THE DATE SCOUT ---
data_dir = r"G:\My Drive\00)Quant_Scripts\WRDS Data\Returns\Options" # Update if needed
files = [f for f in os.listdir(data_dir) if f.endswith(".csv")]

print(f"{'File Name':<45} | {'Start Date':<12} | {'End Date':<12}")
print("-" * 75)

for f in files:
    f_path = os.path.join(data_dir, f)
    # Skip the small volume file or processed files if you want
    if "OptionVolume" in f: continue 
    
    start, end = get_csv_date_range(f_path)
    print(f"{f:<45} | {str(start):<12} | {str(end):<12}")

File Name                                     | Start Date   | End Date    
---------------------------------------------------------------------------
OptionsVolatilitySurface_3.csv                | 2024-01-02   | 2024-12-31  
OptionsVolatilitySurface_4.csv                | 2020-01-02   | 2020-12-31  
OptionsVolatilitySurface_2.csv                | 2021-01-04   | 2021-12-31  
OptionsVolatilitySurface_1.csv                | 2025-01-02   | 2025-08-29  
OptionsVolatilitySurface_2024-2025.csv        | 2024-08-29   | 2025-08-29  
v1zilpeskdggmanu.csv                          | 2023-08-29   | 2024-08-29  
nbbtnjkbfwepkxdq.csv                          | 2022-08-29   | 2023-08-29  
OptionsVolatilitySurface1996-2025_Filtered.csv | 1996-01-04   | 2025-08-29  


In [8]:
# Save this in a new cell in your Data Forge notebook

# 1. Setup - Only target the big file
target_file = "OptionsVolatilitySurface1996-2025_Filtered.csv"
file_path = os.path.join(RAW_DATA_PATH, target_file)
output_parquet = "SPX_Master_1996_2025.parquet"

print(f"🕵️ Inspecting the Golden Ticket: {target_file}...")

# 2. Run the Refinery on JUST this file
# We reuse the logic but strictly for one file to save time
if os.path.exists(file_path):
    # Process
    df_clean = process_and_filter([file_path], target_ticker='SPX')
    
    if df_clean is not None and not df_clean.empty:
        print(f"✅ Success! Found {len(df_clean):,} rows for SPX.")
        print(f"Dates covered: {df_clean['date'].min()} to {df_clean['date'].max()}")
        
        # Save
        df_clean.to_parquet(output_parquet, index=False)
        print(f"💾 Saved to {output_parquet}. You can now delete the CSVs!")
    else:
        print("❌ Bad News: The filtered file exists but had no 'SPX' data.")
        print("You might need to download the missing 'Jan 2022 - Aug 2022' block from WRDS.")
else:
    print(f"❌ File not found: {target_file}")

🕵️ Inspecting the Golden Ticket: OptionsVolatilitySurface1996-2025_Filtered.csv...

Starting Extraction for ticker: SPX
Processing OptionsVolatilitySurface1996-2025_Filtered.csv...
  > Scanned chunk 0... Found 0 rows so far.
  > Scanned chunk 10... Found 4545 rows so far.
  > Scanned chunk 20... Found 9081 rows so far.
  > Scanned chunk 30... Found 13581 rows so far.
  > Scanned chunk 40... Found 18117 rows so far.
  > Scanned chunk 50... Found 20385 rows so far.
  > Scanned chunk 60... Found 24912 rows so far.
  > Scanned chunk 70... Found 29448 rows so far.
  > Scanned chunk 80... Found 31716 rows so far.
  > Scanned chunk 90... Found 33984 rows so far.
  > Scanned chunk 100... Found 36252 rows so far.
  > Scanned chunk 110... Found 40770 rows so far.
  > Scanned chunk 120... Found 43038 rows so far.
  > Scanned chunk 130... Found 45306 rows so far.
  > Scanned chunk 140... Found 47574 rows so far.
  > Scanned chunk 150... Found 49833 rows so far.
  > Scanned chunk 160... Found 52092

C:\Users\Admin\AppData\Local\Temp\ipykernel_58852\79346750.py:46: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  if 'date' in full_df.columns: pd.to_datetime(full_df['date'], format='%Y%m%d', errors='ignore')


KeyError: 'strike_price'

In [9]:
# Save this in a new cell in your Data Forge notebook
import pandas as pd
import os

# Point this to your 2024 file (which we renamed earlier)
# If you haven't renamed it yet, use the original name: "OptionsVolatilitySurface_3.csv"
file_path = os.path.join(r"G:\My Drive\00)Quant_Scripts\WRDS Data\Returns\Options", "Options_Raw_2024_FullYear.csv")

# Fallback if you didn't run the rename script yet
if not os.path.exists(file_path):
    file_path = os.path.join(r"G:\My Drive\00)Quant_Scripts\WRDS Data\Returns\Options", "OptionsVolatilitySurface_3.csv")

print(f"🕵️ Inspecting columns for: {os.path.basename(file_path)}")

try:
    # Read just the header (0 rows)
    df_head = pd.read_csv(file_path, nrows=0)
    
    print("\n--- COLUMNS FOUND ---")
    for col in df_head.columns:
        print(f"- {col}")
        
    print("\n--- DIAGNOSIS ---")
    if 'strike_price' in [c.lower() for c in df_head.columns]:
        print("✅ GOOD NEWS: 'strike_price' exists! You can use this file.")
    elif 'strike' in [c.lower() for c in df_head.columns]:
        print("✅ GOOD NEWS: Found 'strike' column (just need to rename it).")
    else:
        print("❌ BAD NEWS: No strike column found. This is likely the Volatility Surface dataset.")
        print("👉 ACTION: You will need to download 'Option Prices' from WRDS.")

except Exception as e:
    print(f"Error reading file: {e}")

🕵️ Inspecting columns for: Options_Raw_2024_FullYear.csv

--- COLUMNS FOUND ---
- secid
- date
- days
- delta
- impl_volatility
- impl_strike
- impl_premium
- dispersion
- cp_flag
- cusip
- ticker
- sic
- index_flag
- exchange_d
- class
- issue_type
- industry_group

--- DIAGNOSIS ---
❌ BAD NEWS: No strike column found. This is likely the Volatility Surface dataset.
👉 ACTION: You will need to download 'Option Prices' from WRDS.
